In [ ]:
!pip install python-docx openpyxl


In [ ]:
from openpyxl import load_workbook
from docx import Document
import re
import io
from google.colab import files

def find_sentences_with_word_and_pattern(doc, word, pattern):
    found_sentences = []

    for paragraph in doc.paragraphs:
        if word in paragraph.text:
            matches = pattern.findall(paragraph.text)
            found_sentences.extend(matches)

    return found_sentences

# 파일 업로드
uploaded_files = files.upload()
excel_filename = next((name for name in uploaded_files if name.endswith('.xlsx')), None)
word_filename = next((name for name in uploaded_files if name.endswith('.docx')), None)

# 파일 로드
if excel_filename and word_filename:
    wb = load_workbook(filename=io.BytesIO(uploaded_files[excel_filename]))
    sheet = wb.active
    doc = Document(io.BytesIO(uploaded_files[word_filename]))

    pattern = re.compile(r':(.*?)[A-Z]')

    # A열의 단어를 읽고 워드 문서에서 해당 단어가 포함된 ':'와 첫 번째 영문자 사이의 문장 찾기
    for row in range(2, sheet.max_row + 1):
        word = sheet.cell(row=row, column=1).value
        if word:
            sentences = find_sentences_with_word_and_pattern(doc, word, pattern)
            if sentences:
                # 첫 번째 일치하는 문장을 B열에 작성
                sheet.cell(row=row, column=2).value = ' '.join(sentences)

    # 파일 저장
    updated_excel_filename = 'updated_' + excel_filename
    wb.save(updated_excel_filename)
    files.download(updated_excel_filename)
else:
    print("필요한 파일이 모두 업로드되지 않았습니다.")


In [ ]:
!pip install openpyxl


# 새 섹션

In [ ]:
from openpyxl import load_workbook
import re

def remove_text_before_and_including_colon(text):
    # ':'를 포함하여 그 앞의 모든 문자를 제거합니다.
    return re.sub(r'^.*?:', '', text)

# 업데이트 완료된 엑셀 파일 업로드
from google.colab import files
uploaded = files.upload()
excel_filename = next(iter(uploaded))

# 엑셀 파일 로드
wb = load_workbook(filename=excel_filename)
sheet = wb.active

# B열의 데이터 정리
for row in range(2, sheet.max_row + 1):  # 1행은 헤더라고 가정하고 2행부터 시작
    cell_value = sheet.cell(row=row, column=2).value
    if cell_value:
        # ':'와 그 앞의 문자 제거
        cleaned_text = remove_text_before_and_including_colon(cell_value)
        sheet.cell(row=row, column=2).value = cleaned_text

# 파일 저장
wb.save('cleaned_' + excel_filename)

# 파일 다운로드
files.download('cleaned_' + excel_filename)
